In [8]:
# import jsdview_base
# import jsdcontroller
from jsdconfig import JSDConfig
from jsdcontroller import JSDController
from jsdmodel import JSDTableModel
from jsdview_base import JsdViewBase

# import importlib
# importlib.reload(jsdview_base)
# importlib.reload(jsdcontroller)

config = JSDConfig()
jsd_view_base = JsdViewBase()
data_source_list = config.data['data sources']
for data_source in data_source_list:
    print(data_source)
    jsd_view_base.open_excel_file(data_source)
# w = JsdWindow(data_source_list)  # Note: We should have the controller populate this once the tablemodel is loaded
jsd_table_model = JSDTableModel(data_source_list, config.data.get('custom age ranges', None))
jsd_controller = JSDController(jsd_view_base, jsd_table_model, config)


{'name': 'MIDRC', 'description': 'MIDRC Excel File', 'data type': 'file', 'filename': 'MIDRC Open A1 and R1 - cumulative by batch.xlsx', 'remove column name text': ['(CUSUM)']}
{'name': 'CDC', 'description': 'CDC Excel File', 'data type': 'file', 'filename': 'CDC_COVIDpos - cumulative by month.xlsx'}
{'name': 'Census', 'description': 'Census Excel File', 'data type': 'file', 'filename': 'Census_all.xlsx', 'date': '2020-01-01'}
{'name': 'MIDRC COVID+', 'description': 'MIDRC COVID+ Excel File', 'data type': 'file', 'filename': 'MIDRC Open A1 and R1 COVIDpos only - cumulative by batch.xlsx', 'remove column name text': ['(CUSUM)']}


In [23]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from ipywidgets import VBox, HBox, Label
from jsdcontroller import JSDController  # Assuming the backend JSDController is accessible
from PySide6.QtCore import Signal, QObject

# Define the data selection group box
class DataSelectionGroupBox(QObject):
    category_changed = Signal(str)
    file_selection_changed = Signal()

    def __init__(self, jsd_controller):
        super().__init__()
        self.jsd_controller = jsd_controller
        self.file_comboboxes = []
        self.category_combobox = widgets.Dropdown(
            options=[],
            value=None,
            description='Attribute:',
            disabled=True,
        )
        self.layout = VBox()
        self._setup_layout()
        self._initialize_data_sources()

        # Connect signals
        self.category_combobox.observe(self.on_category_changed, names='value')
        for combobox in self.file_comboboxes:
            combobox.observe(self.on_file_selection_changed, names='value')

    def _setup_layout(self):
        self.layout.children = [self.category_combobox]

    def _initialize_data_sources(self):
        # Initialize file comboboxes based on data sources from JSDController
        data_sources = self.jsd_controller.jsd_model.data_sources
        for index, data_source_key in enumerate(list(data_sources.keys())[:2]):
            combobox = widgets.Dropdown(
                options=list([ds_key for ds_key in data_sources.keys()]),
                value=data_source_key,
                description=f'Data Source {index + 1}:',
                disabled=False
            )
            self.file_comboboxes.append(combobox)
            self.layout.children += (combobox,)
            combobox.observe(self.on_file_selection_changed, names='value')

        # Initialize category combobox based on the current data selection
        if self.file_comboboxes:
            self.update_category_combobox()

    def update_category_combobox(self):
        # Update category combobox based on selected data sources
        selected_sheets = [combobox.value for combobox in self.file_comboboxes]
        common_categories = set.intersection(*[set(self.jsd_controller.jsd_model.data_sources[combobox.value].sheets.keys()) for combobox in self.file_comboboxes])
        self.category_combobox.options = list(common_categories)
        self.category_combobox.disabled = False

    def on_category_changed(self, change):
        self.category_changed.emit(change['new'])

    def on_file_selection_changed(self, change):
        self.file_selection_changed.emit()
        self.update_category_combobox()

    def display(self):
        return self.layout

# Create instance of DataSelectionGroupBox
data_selection_groupbox = DataSelectionGroupBox(jsd_controller)

# Define a function to plot the timeline chart
def plot_timeline_chart():
    print("Plotting timeline chart...")
    plt.figure(figsize=(10, 6))
    category = data_selection_groupbox.category_combobox.value
    for combobox in data_selection_groupbox.file_comboboxes:
        if combobox.value:
            try:
                print(f"Processing: {combobox.value}")
                file_data = jsd_controller.jsd_model.data_sources[combobox.value].sheets[category].df
                print(file_data.head())  # Debug to see if file_data is accessible
                if file_data.empty:
                    print(f"No data available for {combobox.value} and category {category}. Skipping.")
                    continue  # Skip empty data
                sns.lineplot(data=file_data, x=file_data.index, y=category, label=f'{combobox.value}: {category}')
            except Exception as e:
                print(f"An error occurred while processing {combobox.value}: {e}")
    print('done with this step')
    plt.xlabel('Index')
    plt.ylabel('Values')
    plt.title('JSD Timeline Chart')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.close()

# Define button to trigger the timeline chart plotting
plot_button = widgets.Button(description='Plot Timeline Chart')
output_timeline = widgets.Output()

def on_plot_button_clicked(b):
    with output_timeline:
        output_timeline.clear_output(wait=True)
        plot_timeline_chart()

plot_button.on_click(on_plot_button_clicked)

# Connect the category change signal to automatically update the chart
data_selection_groupbox.category_changed.connect(on_plot_button_clicked)

# Display the widgets
display(data_selection_groupbox.display(), plot_button, output_timeline)


Button(description='Plot Timeline Chart', style=ButtonStyle())

Output()